<a href="https://colab.research.google.com/github/ancestor9/2025_Fall_AI-Model-Operations-MLOps/blob/main/week09/Get_started_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Google LLC.

# 새 섹션

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Getting started with Gemini models

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Get_started.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

The new **[Google Gen AI SDK](https://github.com/googleapis/python-genai)** provides a unified interface to [Gemini models](https://ai.google.dev/gemini-api/docs/models) through both the [Gemini Developer API](https://ai.google.dev/gemini-api/docs) and the Gemini API on [Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/overview). With a few exceptions, code that runs on one platform will run on both. This notebook uses the Developer API.

This notebook will walk you through:

* [Installing and setting-up](Get_started.ipynb#scrollTo=Mfk6YY3G5kqp) the Google GenAI SDK
* [Text](Get_started.ipynb#scrollTo=6TYNPrNvQ8ue) and [multimodal](#scrollTo=yww-vrxmRiIy) prompting
* Counting [tokens](Get_started.ipynb#scrollTo=_9B8pb7tv_Cx)
* Setting system instructions
* Configuring [safety filters](Get_started.ipynb#scrollTo=HTAnYx_bbxPk)
* Initiating a [multi-turn chat](Get_started.ipynb#scrollTo=HTAnYx_bbxPk)
* [Controlling generated output](Get_started.ipynb#scrollTo=nyZMoM6tgnTA)
* Using [function calling](Get_started.ipynb#scrollTo=Rl-y9SZywD0s)
* Generating a [content stream](Get_started.ipynb#scrollTo=uQfLCxfQtPTg) and sending [asynchronous](Get_started.ipynb#scrollTo=plCtEIaHuv96) requests
* Using [file uploads](Get_started.ipynb#scrollTo=enBhuaIk3KYa)
* Using [context caching](Get_started.ipynb#scrollTo=oTgeR3_9wN5J)
* Generating [text embeddings](Get_started.ipynb#scrollTo=sXNCRn8Wx71d)

More details about this new SDK on the [documentation](https://ai.google.dev/gemini-api/docs/sdks).

## Setup

### Install SDK

Install the SDK from [PyPI](https://github.com/googleapis/python-genai).

In [ ]:
%pip install -U -q 'google-genai>=1.0.0'

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GEMINI_API_KEY`. If you don't already have an API key or you aren't sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [ ]:
from google.colab import userdata

GEMINI_API_KEY = userdata.get('gemini-key')

### Initialize SDK client

With the new SDK, now you only need to initialize a client with you API key (or OAuth if using [Vertex AI](https://cloud.google.com/vertex-ai)). The model is now set in each call.

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GEMINI_API_KEY)

### Choose a model

Select the model you want to use in this guide. You can either select one from the list or enter a model name manually. Keep in mind that some models, such as the 2.5 ones are thinking models and thus take slightly more time to respond. For more details, you can see [thinking notebook](./Get_started_thinking.ipynb) to learn how to switch the thinking off.

For a full overview of all Gemini models, check the [documentation](https://ai.google.dev/gemini-api/docs/models/gemini).

In [ ]:
MODEL_ID = "gemini-2.5-flash" # @param ["gemini-2.5-flash-lite", "gemini-2.5-flash-lite-preview-09-2025", "gemini-2.5-flash", "gemini-2.5-flash-preview-09-2025", "gemini-2.5-pro"] {"allow-input":true, isTemplate: true}

## Send text prompts

Use the `generate_content` method to generate responses to your prompts. You can pass text directly to `generate_content` and use the `.text` property to get the text content of the response. Note that the `.text` field will work when there's only one part in the output.

In [ ]:
from IPython.display import Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents="What's the largest planet in our solar system?"
)

Markdown(response.text)

The largest planet in our solar system is **Jupiter**.

### Upload a PDF file

This PDF page is an article titled [Smoothly editing material properties of objects](https://research.google/blog/smoothly-editing-material-properties-of-objects-with-text-to-image-models-and-synthetic-data/) with text-to-image models and synthetic data available on the Google Research Blog.


In [ ]:
import requests
import pathlib # Import pathlib

PDF = "https://storage.googleapis.com/generativeai-downloads/data/Smoothly%20editing%20material%20properties%20of%20objects%20with%20text-to-image%20models%20and%20synthetic%20data.pdf"  # @param {type: "string"}
pdf_bytes = requests.get(PDF).content

pdf_path = pathlib.Path('article.pdf')
pdf_path.write_bytes(pdf_bytes)

6695391

In [ ]:
import requests
import pathlib
import time
from IPython.display import Markdown

# --- 2. 파일 업로드 (기존 코드) ---
print("파일 업로드 시작...")
# file_upload = client.files.upload(file=pdf_path)
# file_upload = client.files.upload(file="/content/article.pdf") # Added file= and corrected path
file_upload = client.files.upload(file="/content/데이터전처리_예측모형.pdf") # Added file= and corrected path

print(f"파일 업로드 완료. 이름: {file_upload.name}, 상태: {file_upload.state}")


# --- 3. (추가된 해결 로직) 파일 상태 확인 및 대기 함수 ---
def wait_for_file_active(client, file_name, timeout=300, interval=5):
    """
    파일의 상태가 'ACTIVE'가 될 때까지 기다립니다.
    """
    start_time = time.time()
    print(f"파일 '{file_name}'의 처리 완료를 기다리는 중...")

    while time.time() - start_time < timeout:
        # 파일 정보를 다시 가져옵니다.
        file = client.files.get(name=file_name)

        if file.state.name == 'ACTIVE':
            print("✅ 파일 처리 완료! 상태: ACTIVE")
            return file
        elif file.state.name == 'FAILED':
            raise Exception(f"❌ 파일 처리 실패! 상태: FAILED. 파일을 삭제하고 다시 업로드해 보세요.")

        # ACTIVE가 아니면 잠시 기다립니다.
        print(f"현재 상태: {file.state.name}. {interval}초 후 재확인...")
        time.sleep(interval)

    raise TimeoutError(f"⏰ 파일 처리 시간 초과 ({timeout}초).")

# --- 4. 파일 상태가 ACTIVE가 될 때까지 대기 (해결 로직 적용) ---
try:
    active_file = wait_for_file_active(client, file_upload.name)
except (TimeoutError, Exception) as e:
    print(f"오류 발생: {e}")
    # 파일 처리가 실패하거나 시간 초과된 경우, 아래 generate_content를 실행하지 않고 종료
    # (선택적) 에러 발생 시 파일 삭제 로직 추가
    # client.files.delete(name=file_upload.name)
    exit()


# --- 5. 모델 호출 (수정된 코드: ACTIVE 상태의 파일을 사용) ---
print("\n모델에게 요약 요청...")
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        active_file, # ACTIVE 상태가 확인된 파일 사용
        "Can you summarize this file as a bulleted list?",
    ]
)

Markdown(response.text)

파일 업로드 시작...
파일 업로드 완료. 이름: files/5ediawxameek, 상태: FileState.PROCESSING
파일 'files/5ediawxameek'의 처리 완료를 기다리는 중...
현재 상태: PROCESSING. 5초 후 재확인...
✅ 파일 처리 완료! 상태: ACTIVE

모델에게 요약 요청...


Here's a summary of the provided document in a bulleted list:

*   **Research Goal:** To develop a high-performance hybrid deep learning model for predicting aflatoxin occurrence by integrating diverse data sources, effectively addressing severe data imbalance, and establishing a technical foundation for a national food hazard prediction system.

*   **Data Collection:**
    *   **LIMS Data (2015-2025):** From the Ministry of Food and Drug Safety, including inspection results, manufacturing/import details, product names, inspection types (initially 9 types, 40 variables).
    *   **External Climate Data (2015-2025):** Daily agricultural weather observations (76 variables like temperature, humidity, wind speed, precipitation, sunshine, soil moisture).

*   **Data Preprocessing:**
    *   **Initial Cleaning:** Removed irrelevant LIMS records (e.g., "Confirmed after re-testing," "Livestock products," "Dextrin/Mixed preparations"), reducing data from 443,824 to 337,248 records.
    *   **Food Type Classification:** Standardized 1,177 unique product names into 39 categories using an official classification table and Gemini AI for unclassified items.
    *   **Missing Value Handling:**
        *   LIMS: Removed columns/rows with high missing rates (e.g., 'Origin Country Code').
        *   Climate: Removed 'Sunshine Time' (79% missing), imputed other missing values using a 60-day time series interpolation (backward/forward fill, then 0 for ends), successfully reducing 332,419 missing values to 0.
    *   **Downsizing for Class Imbalance:** Removed combinations of inspection purpose and food type with a 0% unsuitability rate historically (e.g., "Inspection Order" for "Agricultural Processed Foods"). This reduced the dataset by 11,227 records (3.33%) to 326,021, and slightly increased the overall unsuitability ratio from 1.17% to 1.21%.
    *   **Final Dataset:** 326,021 records with 548 columns (544 input variables, 4 output variables).

*   **Exploratory Data Analysis (EDA):**
    *   **High-Risk Food Types:** Identified "Nutritional Components" (50% unsuitability), "Jams" (8.7%), "Starchy Roots" (6.25%), and "Edible Oils" (5.41%) as having the highest aflatoxin unsuitability rates.
    *   **High-Risk Inspection Purposes:** "Seizure/Confiscation" (8.77%) and "Manufacturing/Import Permit" (5.83%) showed the highest unsuitability rates.
    *   **Insight:** Suggested that inspection resources could be more efficiently allocated by focusing on high-risk inspection purposes and food types.

*   **Feature Engineering Experiment:**
    *   Generated three new features based on domain knowledge (days with optimal temperature, low humidity, continuous non-precipitation).
    *   **Result:** The models with these engineered features showed *lower* performance (Precision, F1-Score) compared to the original dataset, suggesting these specific features did not improve predictive power in this context.

*   **Model Design (Hybrid Deep Learning Model):**
    *   **Architecture:** A "Hybrid LSTM + Attention + FNN Model Architecture" with two parallel branches to handle heterogeneous data (Temporal Fusion Transformer principles):
        *   **Time-Series Branch:** Processes climate data using Bidirectional LSTM layers and an Attention layer to capture long-term dependencies and important time steps.
        *   **Tabular Branch:** Processes categorical LIMS data using Embedding layers and Dense (Fully Connected) Neural Networks to learn complex interactions.
        *   **Output:** Concatenates the outputs of both branches, followed by a final Dense layer with a Sigmoid activation function for binary classification (predicting unsuitability probability).
    *   **Imbalance Handling in Model Training:** Used SMOTE (Synthetic Minority Over-sampling Technique) applied *only* to the training data, and Focal Loss (instead of Binary Cross-Entropy) as the loss function to give more weight to the minority (unsuitable) class.

*   **Experimental Setup and Evaluation:**
    *   **Two Data Scenarios:**
        *   **Experiment 1:** Used the original highly imbalanced dataset (1.17% unsuitable).
        *   **Experiment 2:** Artificially increased the "unsuitable" rate to 6.49% to test the model's upper performance limits.
    *   **Metrics:** Precision, Recall, F1-Score, AP (Average Precision), and ROC-AUC were used to evaluate performance, with an emphasis on metrics suitable for imbalanced datasets.

*   **Experimental Results (Performance Analysis):**
    *   **Experiment 1 (Original Imbalance):** Achieved high overall accuracy (0.90) but very low performance for the minority class (Precision=0.05, Recall=0.38, F1-Score=0.08), indicating the model was largely predicting the majority class ("suitable").
    *   **Experiment 2 (Adjusted Imbalance):** Showed significant improvement for the minority class (Precision=0.15, Recall=0.56, F1-Score=0.23), demonstrating better identification capability, though still with relatively low precision due to increased false positives for the majority class.
    *   **Calibration:** Both models exhibited calibration issues (under-confidence and over-confidence depending on the predicted probability range).
    *   **Lift Curve:** Both models showed good "lift" (3.43-3.59 times better than random for the top 10% predictions), indicating usefulness for targeting high-risk items.
    *   **Predicted Probability Distribution:** Model 2 showed better separation of predicted probabilities between the two classes.

*   **Final Conclusions and Recommendations:**
    *   The severe data imbalance is the fundamental challenge.
    *   The developed hybrid LSTM-Attention model successfully integrates heterogeneous data, but further threshold adjustment is crucial for practical application, especially to optimize Recall for the minority class.
    *   Calibration techniques (e.g., Platt Scaling, Isotonic Regression) are recommended to improve the reliability of predicted probabilities.

*   **Future Plans:**
    *   **Data Expansion:** Integrate real-time climate data, detailed production area information, and global hazard data.
    *   **Advanced ML Techniques:** Explore Transformer-based models for better contextual understanding.
    *   **Cost-Sensitive Learning:** Implement cost-sensitive learning directly into the loss function to prioritize reducing False Negatives (predicting suitable when it's unsuitable), which is critical in food safety.
    *   **Multi-Hazard Prediction:** Expand the system to predict multiple priority hazards beyond aflatoxin.

*   **Proposed Aflatoxin Prediction & Alert System:**
    *   Predicts unsuitability probabilities 3, 7, and 10 days in advance.
    *   Employs threshold adjustment to minimize False Negatives and maximize Recall.
    *   Defines alert levels (Green: Concern, Yellow: Caution, Orange: Warning) based on predicted probabilities.
    *   Visualizes warnings geographically and over time via web and mobile apps for public access, contributing to public health and food safety.